# 🌩️ Cloudburst Prediction Prototype (Noida)
This notebook processes **ERA5, IMERG, DEM** data and builds a baseline ML model for cloudburst detection.

First Env check will perform..

In [ ]:

import sys
print("✅ Python executable in use:")
print(sys.executable)
print("✅ Python version:")

In [ ]:

import os
env_name = os.environ.get("CONDA_DEFAULT_ENV", "Unknown")
print(f"✅ Active Conda environment: {env_name}")


In [ ]:
print("=== Checking packages for cloudburst_data (preprocessing) ===")
try:
    import numpy, pandas, xarray, rasterio, geopandas, fiona, shapely
    from osgeo import gdal

    print("✅ numpy:", numpy.__version__)
    print("✅ pandas:", pandas.__version__)
    print("✅ xarray:", xarray.__version__)
    print("✅ rasterio:", rasterio.__version__)
    print("✅ geopandas:", geopandas.__version__)
    print("✅ fiona:", fiona.__version__)
    print("✅ shapely:", shapely.__version__)
    print("✅ GDAL:", gdal.__version__)
except Exception as e:
    print("⚠️ Some preprocessing libraries not available:", e)


In [ ]:

print("=== Checking DEM tools ===")
try:
    import richdem
    print("✅ RichDEM available")
except ImportError:
    try:
        import whitebox
        print("⚠️ RichDEM not found, but Whitebox available")
    except ImportError:
        print("❌ Neither RichDEM nor Whitebox available")


In [ ]:

print("=== Checking packages for cloudburst_ml (ML/DL) ===")
try:
    import sklearn, xgboost
    print("✅ scikit-learn:", sklearn.__version__)
    print("✅ xgboost:", xgboost.__version__)
except Exception as e:
    print("⚠️ ML libraries missing:", e)

try:
    import torch
    print("✅ PyTorch:", torch.__version__, "CUDA available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("   GPU:", torch.cuda.get_device_name(0))
except Exception as e:
    print("⚠️ PyTorch not available:", e)

try:
    import tensorflow as tf
    print("✅ TensorFlow:", tf.__version__, "GPU available:", tf.test.is_gpu_available())
except Exception as e:
    print("⚠️ TensorFlow not available:", e)


In [ ]:

# === 1. Setup ===
import os
import numpy as np
import pandas as pd
import xarray as xr
import rasterio
import richdem as rd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier

# Paths
ERA5_PATH_instant = "data/data_raw/era5/data_stream-oper_stepType-instant.nc"
ERA5_PATH_accum = "data/data_raw/era5/data_stream-oper_stepType-accum.nc"
IMERG_PATH = "data/data_raw/imerg/3B-HHR.MS.MRG.3IMERG.20240701-S000000-E002959.0000.V07B.HDF5.nc4"
DEM_PATH = "data/data_raw/dem/noida.tif"

os.makedirs("data/data_processed", exist_ok=True)


This sub-step will verify ERA5 file

In [ ]:
import xarray as xr

# Change path to your file
ERA5_file_instant = ERA5_PATH_instant
ERA5_file_accum = ERA5_PATH_accum

ds = xr.open_dataset(ERA5_file_instant)

# Quick summary
print("\n🔎 Dataset Info:")

print("📂 Global Attributes:")
print(ds.attrs)

print("\n📏 Dimensions:")
print(ds.dims)

print("\n📊 Variables:")
print(list(ds.data_vars))

print(ds)

ds.close()
ds = xr.open_dataset(ERA5_file_accum)

print("📂 Global Attributes:")
print(ds.attrs) 

print("\n📊 Variables:")
print(list(ds.data_vars))
ds.close()


This sub-step will verify IMERG data

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

# Change to your IMERG file path
file_path = IMERG_PATH

# Open dataset
ds = xr.open_dataset(file_path)

# --- Quick details ---
print("📂 Global Attributes:")
print(ds.attrs)

print("\n📏 Dimensions:")
print(ds.dims)

print("\n📊 Variables:")
print(list(ds.data_vars))

print("\n🔎 Dataset Info:")
print(ds)

# --- Quick plot of precipitationCal ---
if "precipitationCal" in ds.data_vars:
    var = ds["precipitationCal"].isel(time=0)  # pick first time step
    plt.figure(figsize=(10, 5))
    var.plot(cmap="Blues", cbar_kwargs={"label": "Rainfall (mm/hr)"})
    plt.title("IMERG PrecipitationCal Snapshot (time=0)")
    plt.show()
else:
    print("\n⚠️ 'precipitationCal' variable not found in this file.")

ds.close()


This sub-step will verify DEM data

In [ ]:
import rasterio

with rasterio.open(DEM_PATH) as dem:
    print("DEM bounds:", dem.bounds)
    print("Resolution:", dem.res)
    print("Shape:", dem.height, "x", dem.width)

In [ ]:
# === 2. Load ERA5 ===
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import richdem as rd

# Paths to downloaded files
instant_file = ERA5_PATH_instant
accum_file   = ERA5_PATH_accum

# Open datasets
ds_instant = xr.open_dataset(instant_file)
ds_accum   = xr.open_dataset(accum_file)

# Merge them
era5 = xr.merge([ds_instant, ds_accum])

# Now select all variables
era5_df = era5[["t2m","d2m","tp","cape","sp"]].to_dataframe().reset_index()

# Rename time column if needed
if "valid_time" in era5_df.columns:
    era5_df = era5_df.rename(columns={"valid_time": "time"})

# Convert cftime to standard datetime
era5_df["time"] = era5_df["time"].apply(lambda x: pd.Timestamp(x.isoformat()))

era5_df.head()



In [ ]:
# === 3. Load IMERG ===
imerg = xr.open_dataset(IMERG_PATH)
print(imerg)

# Resample to hourly mean
imerg_hourly = imerg.resample(time="1h").mean()
imerg_hourly["cloudburst"] = (imerg_hourly["precipitation"] >= 50).astype(int)

# Convert to DataFrame
imerg_df = imerg_hourly[["precipitation","cloudburst"]].to_dataframe().reset_index()

# Convert cftime to standard datetime
imerg_df["time"] = imerg_df["time"].apply(lambda x: pd.Timestamp(x.isoformat()))

imerg_df.head()

In [ ]:
# === 4. Load DEM ===
with rasterio.open(DEM_PATH) as src:
    dem_array = src.read(1)
    bounds = src.bounds
    res = src.res
    print("DEM bounds:", bounds, "Resolution:", res)

dem_rd = rd.LoadGDAL(DEM_PATH)
slope_array = rd.TerrainAttribute(dem_rd, attrib='slope_degrees')

# Quick plots
fig, axes = plt.subplots(1,2, figsize=(10,4))
axes[0].imshow(dem_array, cmap='terrain')
axes[0].set_title("Elevation (m)")
axes[1].imshow(slope_array, cmap='inferno')
axes[1].set_title("Slope (degrees)")
plt.show()


In [ ]:
# === 5. Merge ERA5 + IMERG ===
merged = pd.merge(era5_df, imerg_df, on="time", how="inner")

# Add average DEM features (for Noida region)
merged["elevation"] = np.nanmean(dem_array)
merged["slope"] = np.nanmean(slope_array)

print(merged.head())
print("Dataset shape:", merged.shape)

# Save to CSV
merged.to_csv("data/data_processed/noida_training_dataset.csv", index=False)
print("✅ Training dataset saved: data/data_processed/noida_training_dataset.csv")

In [ ]:
# === 6. Train Baseline ML Model ===
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Load dataset (with time column preserved for plotting in Step 7)
merged = pd.read_csv("data/data_processed/noida_training_dataset.csv")

# Ensure time is datetime
if "time" in merged.columns:
    merged["time"] = pd.to_datetime(merged["time"], errors="coerce")

# Features and target
X = merged[["t2m", "d2m", "tp", "cape", "sp", "elevation", "slope"]].fillna(0).astype(float)
y = merged["cloudburst"].fillna(0).astype(int)

# Train-test split (handle single-class case gracefully)
if y.nunique() > 1:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
else:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

# Train XGBoost classifier (binary classification mode)
model = XGBClassifier(
    eval_metric="logloss",
    use_label_encoder=False,
    objective="binary:logistic",  # Binary classification
    base_score=0.5,
    random_state=42
)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
print("\n=== Model Evaluation ===")
print("Classification Report:\n", classification_report(y_test, y_pred, zero_division=0))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred, labels=[0, 1]))

# Save for Step 7 visualization
model_df = merged.copy()


In [ ]:
# === 7. Interactive Visualization: Actual vs Predicted Cloudburst + Threshold ===
import plotly.express as px
import plotly.graph_objects as go

# Add predicted cloudburst column from model
merged["cloudburst_pred"] = model.predict(X)

# Create a new column for combined status
def combined_status(row):
    if row["cloudburst"] == 1 and row["cloudburst_pred"] == 1:
        return "True Positive"
    elif row["cloudburst"] == 0 and row["cloudburst_pred"] == 0:
        return "True Negative"
    elif row["cloudburst"] == 0 and row["cloudburst_pred"] == 1:
        return "False Positive"
    else:
        return "False Negative"

merged["status"] = merged.apply(combined_status, axis=1)

# Base scatter plot
fig = px.scatter(
    merged,
    x="time",
    y="tp",
    color="status",
    labels={
        "tp": "Rainfall (tp mm/hr)",
        "time": "Time",
        "status": "Cloudburst Prediction Status"
    },
    hover_data=["t2m","d2m","cape","sp","elevation","slope","cloudburst","cloudburst_pred"]
)

# Add threshold line at 50 mm/hr
fig.add_trace(
    go.Scatter(
        x=merged["time"],
        y=[50]*len(merged),
        mode="lines",
        line=dict(color="red", dash="dash"),
        name="Cloudburst threshold (50 mm/hr)"
    )
)

fig.update_traces(marker=dict(size=10, line=dict(width=1, color='DarkSlateGrey')))
fig.update_layout(
    title="Actual vs Predicted Cloudburst Events (Interactive) with Threshold",
    xaxis_title="Time",
    yaxis_title="Rainfall (tp mm/hr)",
    hovermode="closest"
)
fig.show()
